Reconstructing a lineage tree from mutation table

In [1]:
import os
import sys
sys.path.append('path_to_examples')

from utils import run_command

import pandas as pd

In [2]:
project_dir = 'path_to_example'

aquisition_coverage_case = 'n-000004'


Evaluating the tree reconstruction agains a know reference using TreeCMP

In [3]:
import dendropy
tns = dendropy.TaxonNamespace()
sim_tree = dendropy.Tree.get_from_path(os.path.join(project_dir,'simulation.newick'),'newick', taxon_namespace=tns, preserve_underscores=True)
ft_rec_tree = dendropy.Tree.get_from_path(os.path.join(project_dir,'reconstruction.newick'),"newick",taxon_namespace=tns, preserve_underscores=True)

In [5]:
# Clean up leaf labels if they are '_' seperated assuming the last segment is the leaf ID
for n in sim_tree.leaf_node_iter():
    n.taxon.label = n.taxon.label.split('_')[-1]
for n in ft_rec_tree.leaf_node_iter():
    n.taxon.label = n.taxon.label.split('_')[-1]
sim_tree.write_to_path(os.path.join(project_dir,'simulation.idsonly.newick'), 'newick')
ft_rec_tree.write_to_path(os.path.join(project_dir,'reconstruction.idsonly.newick'), 'newick')

In [6]:
def compare(path_simulation_newick, path_reconstructed_newick, path_score_output):

    path_treecmp = 'path_to_treecmp/bin/TreeCmp/bin/treeCmp.jar'
    
    #  Metrics for rooted trees
    metrics = {
        "rooted": "mc rc ns tt mp mt co",
        "unrooted": "ms rf pd qt um"
    }

    cmd = [
        'java', '-jar', path_treecmp,
        '-P', '-N', '-I'
        '-r', path_simulation_newick,
        '-i', path_reconstructed_newick,
        '-o', path_score_output,
        '-d'
    ]

    cmd.extend(metrics['rooted'].split(' '))

    run_command(cmd)
    
    
def get_scores(path_simulation_newick, path_reconstructed_newick, path_score_output):

    compare(path_simulation_newick, path_reconstructed_newick, path_score_output)

    df_metrics = pd.read_csv(
        path_score_output,
        sep='\t',
        nrows=1
    )

    return df_metrics

In [7]:
scores = get_scores(
    os.path.join(project_dir,'simulation.idsonly.newick'),
    os.path.join(project_dir,'reconstruction.idsonly.newick'),
    '/tmp/scores_ft.txt'
)

In [8]:
scores.T

,0
No,1.0000
RefTree,1.0000
Tree,1.0000
RefTree_taxa,55.0000
Tree_taxa,56.0000
Common_taxa,55.0000
MatchingCluster,212.0000
MatchingCluster_toYuleAvg,0.5308
MatchingCluster_toUnifAvg,0.3805
R-F_Cluster,33.5000
